In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from torchvision import utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_train = '/content/drive/MyDrive/BME AI/CNN classification/data2/train/'
path_valid = '/content/drive/MyDrive/BME AI/CNN classification/data2/validation/'
path_test = '/content/drive/MyDrive/BME AI/CNN classification/data2/test/'

## 영상데이터에 대한 전처리과정 정의하기

In [ ]:
train_transforms = transforms.Compose(
      [
          transforms.Grayscale(num_output_channels=3),
          transforms.Resize((224,224)),
          transforms.ToTensor(),
      ]
    )

## ImageFolder를 활용하여 data loader를 생성하기

In [ ]:
batch_size = 32

trainset = torchvision.datasets.ImageFolder(root = path_train,transform=train_transforms)
validset = torchvision.datasets.ImageFolder(root = path_valid,transform=train_transforms)
testset = torchvision.datasets.ImageFolder(root = path_test,transform=train_transforms)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
validloader = torch.utils.data.DataLoader(validset,batch_size=batch_size,shuffle=False)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False)


In [ ]:
tgtnames = trainset.classes
print(tgtnames)

In [ ]:
print(len(trainset),len(validset), len(testset))

In [ ]:
for X, y in trainloader:
  print(X.shape)
  print(y.shape)
  I = X[0][0].numpy()
  plt.figure(dpi=128)
  plt.imshow(I,cmap='gray')
  plt.title(tgtnames[y[0]])
  plt.show()
  break

In [ ]:
pip install timm

In [ ]:
import timm
num_classes = 4
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

citerion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 30  # 학습할 epoch 수입니다.

In [ ]:
## Adam optimizer

# import torch.optim as optim
# import torch.nn as nn
# from torch.optim import lr_scheduler

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# import torchsummary


# torchsummary.summary(model, (3,224,224))

## Model 학습 진행

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
num_classes = 4  # 분류할 클래스의 개수입니다.
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()  # 손실 함수로 CrossEntropyLoss를 사용합니다.
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # 최적화 알고리즘으로 SGD를 사용합니다.
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # learning rate를 조절하기 위한 스케줄러입니다.

num_epochs = 30  # 학습할 epoch 수입니다.

In [ ]:
# 모델 학습

loss_train = []
accs_train = []
accs_valid = []
accs_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # print("-" * 10)
    
    # 학습 모드로 설정
    model.train()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 역전파 + 최적화
        loss.backward()
        optimizer.step()
        
        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # 에폭별 학습 결과 출력
    epoch_loss = running_loss / len(trainset)
    epoch_acc = correct / total
    loss_train.append(epoch_loss)
    accs_train.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    
    # 검증 모드로 설정
    model.eval()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in validloader:
            images = images.to(device)
            labels = labels.to(device)
            
            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # 통계
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # 에폭별 검증 결과 출력
    epoch_loss = running_loss / len(validset)
    epoch_acc = correct / total
    accs_valid.append(epoch_acc)
    print(f"Valid Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    
    # learning rate 갱신
    scheduler.step()

print("Training complete.")




Epoch 1/30
Train Loss: 0.7526 Acc: 0.6885
Valid Loss: 0.5543 Acc: 0.7544
Epoch 2/30
Train Loss: 0.2168 Acc: 0.9148
Valid Loss: 0.2466 Acc: 0.9298
Epoch 3/30
Train Loss: 0.0865 Acc: 0.9623
Valid Loss: 0.4688 Acc: 0.8947
Epoch 4/30
Train Loss: 0.0467 Acc: 0.9846
Valid Loss: 0.1700 Acc: 0.9123
Epoch 5/30
Train Loss: 0.0230 Acc: 0.9930
Valid Loss: 0.1560 Acc: 0.9474
Epoch 6/30
Train Loss: 0.0237 Acc: 0.9958
Valid Loss: 0.3036 Acc: 0.9211
Epoch 7/30
Train Loss: 0.0057 Acc: 0.9986
Valid Loss: 0.1653 Acc: 0.9561
Epoch 8/30
Train Loss: 0.0015 Acc: 1.0000
Valid Loss: 0.1566 Acc: 0.9561
Epoch 9/30
Train Loss: 0.0009 Acc: 1.0000
Valid Loss: 0.1492 Acc: 0.9561
Epoch 10/30
Train Loss: 0.0007 Acc: 1.0000
Valid Loss: 0.1446 Acc: 0.9561
Epoch 11/30
Train Loss: 0.0006 Acc: 1.0000
Valid Loss: 0.1420 Acc: 0.9561
Epoch 12/30
Train Loss: 0.0006 Acc: 1.0000
Valid Loss: 0.1394 Acc: 0.9561
Epoch 13/30
Train Loss: 0.0005 Acc: 1.0000
Valid Loss: 0.1375 Acc: 0.9561
Epoch 14/30
Train Loss: 0.0005 Acc: 1.0000
Vali

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
            
        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)
            
        # 통계
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 테스트 정확도 출력
epoch_acc = correct / total
accs_test.append(epoch_acc)  # 테스트 정확도 추가
print(f"Test Accuracy: {epoch_acc:.4f}")

In [ ]:
print("Training Loss:", loss_train)
print("Training Accuracy:", accs_train)
print("Validation Accuracy:", accs_valid)
print("Test Accuracy:", accs_test)

## 학습과정의 요약

In [ ]:
plt.figure(2,dpi=80)
plt.subplot(121)
plt.plot(loss_train,label='train loss')
plt.legend(loc='upper right')
plt.subplot(122)
plt.plot(accs_train,label='train accuracy')
plt.plot(accs_valid,label='valid accuracy')
# plt.plot(accs_test, label='test accuracy')
plt.legend(loc='lower right')
plt.show()

## Validation set에 대한 개별결과 확인 (Inference)

In [ ]:
from PIL import Image

In [ ]:
N = 51
#validset.imgs[N][0]

In [ ]:
I = Image.open(validset.imgs[N][0])
X = train_transforms(I)
y = validset.targets[N]

print(tgtnames[y])
I


## Validation set에 대한 결과 요약하기

In [ ]:
y_list = np.array([])
y_hat_list = np.array([])
for X,y in validloader:
  y_hat = model(X.to(device))    
  y_hat = y_hat.argmax(dim=1)
  y_list = np.append(y_list,y)
  y_hat_list = np.append(y_hat_list,y_hat.cpu().numpy())

In [ ]:
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)

In [ ]:
print(classification_report(
    y_list,
    y_hat_list,
    target_names=tgtnames))

In [ ]:
cm = confusion_matrix(
    y_list,
    y_hat_list,
#    normalize='true',
)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=tgtnames,
)
disp.plot(ax=plt.subplots(1, 1, facecolor='white')[1])

## Test set에 대한 결과 요약하기

In [ ]:
y_list = np.array([])
y_hat_list = np.array([])
for X,y in testloader:
  y_hat = model(X.to(device))    
  y_hat = y_hat.argmax(dim=1)
  y_list = np.append(y_list,y)
  y_hat_list = np.append(y_hat_list,y_hat.cpu().numpy())

In [ ]:
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)

In [ ]:
print(classification_report(
    y_list,
    y_hat_list,
    target_names=tgtnames))

In [ ]:
cm = confusion_matrix(
    y_list,
    y_hat_list,
#    normalize='true',
)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=tgtnames,
)
disp.plot(ax=plt.subplots(1, 1, facecolor='white')[1])